<img src="images/usm.jpg" width="480" height="240" align="left"/>

# MAT281 - Laboratorio N°02

## Objetivos del laboratorio

* Reforzar conceptos básicos de clasificación.

## Contenidos

* [Problema 01](#p1)


<a id='p1'></a>
## I.- Problema 01


<img src="https://www.xenonstack.com/wp-content/uploads/xenonstack-credit-card-fraud-detection.png" width="360" height="360" align="center"/>


El conjunto de datos se denomina `creditcard.csv` y consta de varias columnas con información acerca del fraude de tarjetas de crédito, en donde la columna **Class** corresponde a: 0 si no es un fraude y 1 si es un fraude.

En este ejercicio se trabajará el problemas de  clases desbalancedas. Veamos las primeras cinco filas dle conjunto de datos:

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [2]:
# cargar datos
df = pd.read_csv(os.path.join("data","creditcard.csv"), sep=";")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
2,10.0,1.249999,-1.221637,0.383930,-1.234899,-1.485419,-0.753230,-0.689405,-0.227487,-2.094011,...,-0.231809,-0.483285,0.084668,0.392831,0.161135,-0.354990,0.026416,0.042422,121.50,0
3,10.0,0.384978,0.616109,-0.874300,-0.094019,2.924584,3.317027,0.470455,0.538247,-0.558895,...,0.049924,0.238422,0.009130,0.996710,-0.767315,-0.492208,0.042472,-0.054337,9.99,0
4,12.0,1.103215,-0.040296,1.267332,1.289091,-0.735997,0.288069,-0.586057,0.189380,0.782333,...,-0.024612,0.196002,0.013802,0.103758,0.364298,-0.382261,0.092809,0.037051,12.99,0


Analicemos el total de fraudes respecto a los casos que nos son fraudes:


In [3]:
# calcular proporciones
df_count = pd.DataFrame()
df_count["fraude"] =["no","si"]
df_count["total"] = df["Class"].value_counts() 
df_count["porcentaje"] = 100*df_count["total"] /df_count["total"] .sum()

df_count

,fraude,total,porcentaje
0,no,50000,99.025588
1,si,492,0.974412


Se observa que menos del 1% corresponde a registros frudulentos. La pregunta que surgen son:

* ¿ Cómo deben ser el conjunto de entrenamiento y de testeo?
* ¿ Qué modelos ocupar?
* ¿ Qué métricas ocupar?

Por ejemplo, analicemos el modelos de regresión logística y apliquemos el procedimiento estándar:

In [4]:
# datos 
y = df.Class
X = df.drop('Class', axis=1)

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)


# Creando el modelo
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# predecir
lr_pred = lr.predict(X_test)

# calcular accuracy
accuracy_score(y_test, lr_pred)

0.9961181969420898

En general el modelo tiene un **accuracy** del 99,9%, es decir, un podría suponer que el modelo predice casi perfectamente, pero eso esta lejos de ser así.  Para ver por qué es necesario seguir los siguientes pasos:

### 1. Cambiar la métrica de rendimiento

El primer paso es comparar con distintas métricas, para eso ocupemos las 4 métricas clásicas abordadas en el curso:
* accuracy
* precision
* recall
* f-score

En este punto deberá poner las métricas correspondientes y comentar sus resultados.

In [5]:
# metrics
y_true =  list(y_test)
y_pred = list(lr.predict(X_test))

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_test, lr_pred))
print('recall:     ',recall_score(y_test, lr_pred))
print('precision:  ',precision_score(y_test, lr_pred))
print('f-score:    ',f1_score(y_test, lr_pred))
print("")


Matriz de confusion:
 
[[12471    16]
 [   33   103]]

Metricas:
 
accuracy:    0.9961181969420898
recall:      0.7573529411764706
precision:   0.865546218487395
f-score:     0.807843137254902



## Comentarios de Resultados 1

En el contexto del problema, como la clase de fraude es minuscula en comparación a la clase no fraude, la métrica acurracy no es un buen estimador de que tan buena fue la predicción (Es una métrica muy suceptible a diferencias grandes en la cantidad de valores de cada clase). Aun cuando la métrica acurracy fue alta (99,6 %), con lo que se dijo anteriormente, conviene mirar las demás métricas para obtener un mejor análisis para este problema. Si miramos la métrica recall:

$$recall(y,\hat{y}) = \dfrac{TP}{TP+FN}$$

El valor que se obtuvo de un 75%, nos estaría de la totalidad de las tajetas con fraude, solo un 75% se predijeron correctamente. Debido a la seriedad de un fraude de una tarjeta bancaria, a mi opinión un indíce de predicción correcta de fraude del 75% no es suficiente para un banco. 

También observando la precision se tiene que de la totalidad de las tarjetas que se predijeron fraude, solo un 86% en verdad lo eran, denuevo, debido a la seriedad de los fraudes, en mi opinión no es suficiente.

### 2. Cambiar algoritmo

El segundo paso es comparar con distintos modelos. Debe tener en cuenta que el modelo ocupaod resuelva el problema supervisado de clasificación.

En este punto deberá ajustar un modelo de **random forest**, aplicar las métricas y comparar con el modelo de regresión logística.

In [6]:
# train model

rfc =  RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1).fit(X_train, y_train) # algoritmo random forest

In [7]:
# metrics

y_true =  list(y_test)
y_pred = list(rfc.predict(X_test)) # predicciones con random forest


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12485     2]
 [   56    80]]

Metricas:
 
accuracy:    0.9954052127069635
recall:      0.5882352941176471
precision:   0.975609756097561
f-score:     0.7339449541284403



## Comentarios de Resultados 2

AL igual que en 1, el acurracy es alto solo por la gran diferencia en la cantidad de cada clase, así que conviene mirar las demás métricas.

En comparación con 1, la métrica recall bajó hasta un 60% denuevo diciendo que solo se predijeron correctamente 60& de las tarjetas con fraude. En mi opinión, aunque en 1 mencioné que era malo que solo se tuviera un 75%, en esta predicción se obtuvo aun más bajo, por lo que preferíria el metodo de 1.

Se observa también que la métrica precision fue de 97-98% muy cercano al 100%, pero como está métrica mide cuantos datos de los que se dijo que eran fraude en realidad lo erán, el hecho que el recall haya sido tan bajo le quita mucha validez al método (esto también se ve reflejado en el f-score que se puede entender como algun tipo de media entre recall y precision).

### 3. Técnicas de remuestreo: sobremuestreo de clase minoritaria

El tercer paso es ocupar ténicas de remuestreo, pero sobre la clase minoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase minoritaria a la clase mayoritaria.

In [8]:
from sklearn.utils import resample

# concatenar el conjunto de entrenamiento
X = pd.concat([X_train, y_train], axis=1)

# separar las clases
not_fraud = X[X.Class==0]
fraud = X[X.Class==1]

# remuestrear  clase minoritaria
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# recombinar resultados
upsampled = pd.concat([not_fraud, fraud_upsampled])

# chequear el número de elementos por clases
upsampled.Class.value_counts()

1    37513
0    37513
Name: Class, dtype: int64

In [9]:
# datos de entrenamiento sobre-balanceados
y_train = upsampled.Class
X_train = upsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [10]:
upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train) # algoritmo de regresion logistica

# metrics

y_true =  list(y_test)
y_pred = list(upsampled.predict(X_test))


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12200   287]
 [   12   124]]

Metricas:
 
accuracy:    0.976313079299691
recall:      0.9117647058823529
precision:   0.30170316301703165
f-score:     0.45338208409506403



## Comentarios de Resultados 3

En este caso notamos una disminución considerable en la métrica precisión, causada por el aumento considerable en la catidad de los datos de la clase fraude. Observamos también que acurracy se mantuvo alto y recall subió, lo que puede ser beneficial para este problema en especifico.
Debido a la gran dierencia que hay en la cantidad de valores de cada clase, al ocupar esta técnica podríamos estar afectando demasiado la veracidad de la clasificación. Al agregar una magnitud tan grande de datos, el peso que tienen los datos originales es muy poco en la clasificación.
Aun así, una ventaja de este metodo es que es más fácil analizar datos balanceados de manera segura, las métricas serán más representativas en datos balanceados.

### 4. Técnicas de remuestreo - Ejemplo de clase mayoritaria

El cuarto paso es ocupar ténicas de remuestreo, pero sobre la clase mayoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase mayoritaria  a la clase minoritaria.

In [11]:
# remuestreo clase mayoritaria
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27) # reproducible results

# recombinar resultados
downsampled = pd.concat([not_fraud_downsampled, fraud])

# chequear el número de elementos por clases
downsampled.Class.value_counts()

1    356
0    356
Name: Class, dtype: int64

In [12]:
# datos de entrenamiento sub-balanceados

y_train = downsampled.Class
X_train = downsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [13]:
undersampled = LogisticRegression(solver='liblinear').fit(X_train, y_train) # modelo de regresi+on logística

# metrics

y_true =  list(y_test)
y_pred = list(undersampled.predict(X_test))


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12216   271]
 [   16   120]]

Metricas:
 
accuracy:    0.9772637249465261
recall:      0.8823529411764706
precision:   0.3069053708439898
f-score:     0.45540796963946867



## Comentarios de Resultados 4

Se puede observar una pequeña disminución en recall en comparación con 3 pero las demás metricas se mantuvieron parecidas.

Este método puede ser muy perjudicial por el hecho de que estamos quitando datos del análisis, en este caso se perdieron más de 30000 datos para disminuir las clase mayoritaria. Esta técnica se podría usar cuando la totalidad de los datos es sifnificativamente mayor a la diferencia entre las clases. Una ventaja al igual que en 3, es que es más hacer un análisis con una clasificación de datos balanceada, las métricas entregan información más represntativa.

### 5. Conclusiones

Para finalizar el laboratorio, debe realizar un análisis comparativo con los disintos resultados obtenidos  en los pasos 1-4. Saque sus propias conclusiones del caso.

Notamos que en todos los casos la métrica acurracy se mantuvo sobre el 97% lo que me hace pensar que no es un muy buen indicador de validez de técnicas. 

Debido al contexto del problema, creo importante ponerle enfasis al estudio de recall, puesto que nos está entregando información de buena oprediccion de fraudes. Con esto, el método 3 fue el obtuvo un mayor valor con un 91%, pero se contrasto con el hecho de agregar una cantidad de casi 40000 datos a la clasificación, así que no veo muy válido el análisis en este caso.

Se observa que en los casos 3 y 4 se obtuvo un error bastante grande reflejado en el f-score, esto en el contexto del problema es bien grave por estar lidiando con fraudes de tarjestas bancarias. Si observamos solo el valor del f-score en los 4 casos, el primero fue el que obtuvo un mayor valor de aprox 81 % y además tenía valores por sobre el 75% para precision y recall por lo que diría que fue el método más correcto entre los 4.

Si tuviera que elegir entre las técnicas 3 y 4, preferiría la técnica 3 puesto que eliminar datos como en la técnica 4 en este caso lo encuentro sumamente perjudicial, se eliminaban casi 30000 datos en la clasificación, perdiendo una gran cantidad de información para un análisis posterior. En cambio en la técnica 4 no quitamos información pero si agregamos demsaidos datos que no estaban inicialmente.

